In [11]:
import boto3
import os

def create_key_pair():
    ec2_client = boto3.client("ec2", region_name='us-east-1')
    key_pair = ec2_client.create_key_pair(KeyName="lab4-key-pair")
    private_key = key_pair["KeyMaterial"]
    with os.fdopen(os.open("lab4_ec2_key.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)

create_key_pair()


In [ ]:
import boto3

USERDATA_SCRIPT = '''#!/bin/bash
cd /home/ubuntu/
git clone https://github.com/danilaSADev/AWS-Lab---Deploy-automatization.git "./cloned-dist/"
cd cloned-dist
sudo apt update
sudo apt install python3-pip --assume-yes
sudo apt install jupyter-notebook --assume-yes
pip3 install -r requirements.txt
pip3 install jupyter-notebook
pip3 install pycairo'''

def create_instance():
    ec2_client = boto3.client("ec2", region_name="us-east-1")
    instances = ec2_client.run_instances(
        ImageId="ami-08c40ec9ead489470",
        MinCount=1,
        MaxCount=1,
        SecurityGroupIds=[
            "sg-05ab746d1e2ef73af"
        ],
        InstanceType="t2.micro",
        KeyName="lab4-key-pair",
        UserData=USERDATA_SCRIPT
    )
    print(instances["Instances"][0]["InstanceId"])

create_instance()


In [4]:
import boto3

id = "i-07d3148832499d71e"

def get_public_ip(instance_id):
    ec2_client = boto3.client("ec2", region_name="us-east-1")

    reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")

    for reservation in reservations:
        for instance in reservation['Instances']:
            print(instance.get("PublicIpAddress"))


get_public_ip(id)

54.210.158.224


In [4]:
import boto3

def get_running_instances():
    ec2_client = boto3.client("ec2", region_name="us-east-1")
    reservations = ec2_client.describe_instances(Filters=[
        {
            "Name": "instance-state-name",
            "Values": ["running"],
        },
        {
            "Name": "instance-type",
            "Values": ["t2.micro"]
        }
    ]).get("Reservations")
    for reservation in reservations:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            instance_type = instance["InstanceType"]
            public_ip = instance["PublicIpAddress"]
            private_ip = instance["PrivateIpAddress"]
            print(f"{instance_id}, {instance_type}, {public_ip}, {private_ip}")

get_running_instances()

i-07d3148832499d71e, t2.micro, 34.227.111.92, 172.31.27.243
i-0b68e8f8efc3d1672, t2.micro, 34.207.92.208, 172.31.81.71
i-0c337bd8f91d8e833, t2.micro, 54.152.91.164, 172.31.89.242
i-08ee6970fb4a4756c, t2.micro, 3.89.124.175, 172.31.95.2


In [7]:
import boto3

id = 'i-07d3148832499d71e'

def stop_instance(instance_id):
    ec2_client = boto3.client("ec2", region_name="us-east-1")
    response = ec2_client.stop_instances(InstanceIds=[instance_id])
    print(response)

stop_instance(id)

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-07d3148832499d71e', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '0c710596-db39-4b62-ad7f-812390eb6fd8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0c710596-db39-4b62-ad7f-812390eb6fd8', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Thu, 17 Nov 2022 17:04:19 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
